# Import Required Libraries
Import the necessary libraries, including snowflake.connector and pandas.

In [1]:
# Import the necessary libraries
import snowflake.connector
import pandas as pd

# Establish Connection to Snowflake
Establish a connection to Snowflake using the snowflake.connector.connect() method.

In [2]:
# Establish Connection to Snowflake

# Define connection parameters
conn_params = {
    "user": "dushantgohri",
    "password": "Dg@17343857148",
    "account": "pwizmma-ka25931",
    "warehouse": "compute_wh",
    "database": "test_datawarehouse",
    "schema": "public"
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**conn_params)

# Verify the connection by fetching the current version of Snowflake
cursor = conn.cursor()
cursor.execute("SELECT current_version()")
version = cursor.fetchone()
print(f"Snowflake version: {version[0]}")

# Close the cursor
cursor.close()

Snowflake version: 9.3.3


True

# Create Tables if Not Exists
Use SQL commands to create tables in Snowflake if they do not already exist.

In [3]:
# Create Tables if Not Exists

# Create a cursor object
cursor = conn.cursor()

# SQL command to create a table if it does not exist
create_train_table_query = """
CREATE TABLE IF NOT EXISTS train_table (
    id INT AUTOINCREMENT,
    feature1 STRING,
    feature2 STRING,
    target FLOAT,
    PRIMARY KEY (id)
)
"""

create_test_table_query = """
CREATE TABLE IF NOT EXISTS test_table (
    id INT AUTOINCREMENT,
    feature1 STRING,
    feature2 STRING,
    target FLOAT,
    PRIMARY KEY (id)
)
"""

# Execute the SQL commands
cursor.execute(create_train_table_query)
cursor.execute(create_test_table_query)

# Commit the transaction
conn.commit()

# Close the cursor
cursor.close()

True

# Insert Data into Tables
Use SQL INSERT commands to insert data into the Snowflake tables.

In [4]:
# Insert Data into Tables

# Create a cursor object
cursor = conn.cursor()

# Sample data to insert into train_table
train_data = [
    (1, 'feature1_value1', 'feature2_value1', 0.5),
    (2, 'feature1_value2', 'feature2_value2', 1.5)
]

# Sample data to insert into test_table
test_data = [
    (1, 'feature1_value1', 'feature2_value1', 0.7),
    (2, 'feature1_value2', 'feature2_value2', 1.7)
]

# SQL command to insert data into train_table
insert_train_query = """
INSERT INTO train_table (id, feature1, feature2, target)
VALUES (%s, %s, %s, %s)
"""

# SQL command to insert data into test_table
insert_test_query = """
INSERT INTO test_table (id, feature1, feature2, target)
VALUES (%s, %s, %s, %s)
"""

# Insert data into train_table
cursor.executemany(insert_train_query, train_data)

# Insert data into test_table
cursor.executemany(insert_test_query, test_data)

# Commit the transaction
conn.commit()

# Close the cursor
cursor.close()

True

# Copy Data into Tables
Use the COPY INTO command to copy data from a file into the Snowflake tables.

In [5]:
# Copy Data into Tables

# Create a cursor object
cursor = conn.cursor()

# SQL command to copy data from a file into train_table
copy_train_query = """
COPY INTO train_table
FROM @%train_table
FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY = '"')
"""

# SQL command to copy data from a file into test_table
copy_test_query = """
COPY INTO test_table
FROM @%test_table
FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY = '"')
"""

# Execute the SQL commands
cursor.execute(copy_train_query)
cursor.execute(copy_test_query)

# Commit the transaction
conn.commit()

# Close the cursor
cursor.close()

True

# Fetch Data from Tables
Use SQL SELECT commands to fetch data from the Snowflake tables and load it into a pandas DataFrame.

In [4]:
# Fetch Data from Tables

# Create a cursor object
cursor = conn.cursor()

# SQL command to fetch data from train_table
fetch_train_query = "SELECT * FROM DA_DS_HOUSING_V2"

# Execute the SQL command and load the data into a pandas DataFrame
train_data = pd.read_sql(fetch_train_query, conn)
print("Train Data:")
print(train_data.head())

# Save the train data in S3, for pipeline to be picked the data

train_data.to_csv('s3://sagemaker-us-east-1-237682134737/input_data/housing_data_v2.csv')

/tmp/ipykernel_6580/555868804.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  train_data = pd.read_sql(fetch_train_query, conn)


Train Data:
      PRICE  AREA  BEDROOMS  BATHROOMS  STORIES  MAINROAD  GUESTROOM  \
0  13300000  7420         4          2        3      True      False   
1  12250000  8960         4          4        4      True      False   
2  12250000  9960         3          2        2      True      False   
3  12215000  7500         4          2        2      True      False   
4  11410000  7420         4          1        2      True       True   

   BASEMENT  HOTWATERHEATING  AIRCONDITIONING  PARKING  PREFAREA  \
0     False            False             True        2      True   
1     False            False             True        3     False   
2      True            False            False        2      True   
3      True            False             True        3      True   
4      True            False             True        2     False   

  FURNISHINGSTATUS  
0        furnished  
1        furnished  
2   semi-furnished  
3        furnished  
4        furnished  


In [5]:
# reading the data
train_data =  pd.read_csv('s3://sagemaker-us-east-1-237682134737/input_data/housing_data_v2.csv')

train_data.head()

,Unnamed: 0,PRICE,AREA,BEDROOMS,BATHROOMS,STORIES,MAINROAD,GUESTROOM,BASEMENT,HOTWATERHEATING,AIRCONDITIONING,PARKING,PREFAREA,FURNISHINGSTATUS
0,0,13300000,7420,4,2,3,True,False,False,False,True,2,True,furnished
1,1,12250000,8960,4,4,4,True,False,False,False,True,3,False,furnished
2,2,12250000,9960,3,2,2,True,False,True,False,False,2,True,semi-furnished
3,3,12215000,7500,4,2,2,True,False,True,False,True,3,True,furnished
4,4,11410000,7420,4,1,2,True,True,True,False,True,2,False,furnished


In [6]:
train_data =  train_data.drop('Unnamed: 0',axis=1)

In [7]:
train_data.head()

,PRICE,AREA,BEDROOMS,BATHROOMS,STORIES,MAINROAD,GUESTROOM,BASEMENT,HOTWATERHEATING,AIRCONDITIONING,PARKING,PREFAREA,FURNISHINGSTATUS
0,13300000,7420,4,2,3,True,False,False,False,True,2,True,furnished
1,12250000,8960,4,4,4,True,False,False,False,True,3,False,furnished
2,12250000,9960,3,2,2,True,False,True,False,False,2,True,semi-furnished
3,12215000,7500,4,2,2,True,False,True,False,True,3,True,furnished
4,11410000,7420,4,1,2,True,True,True,False,True,2,False,furnished


In [6]:
# SQL command to fetch data from test_table
fetch_test_query = "SELECT * FROM test_table"

# Execute the SQL command and load the data into a pandas DataFrame
test_data = pd.read_sql(fetch_test_query, conn)
print("Test Data:")
print(test_data.head())

# Close the cursor
cursor.close()

/tmp/ipykernel_208/31309882.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  train_data = pd.read_sql(fetch_train_query, conn)


Train Data:
   ID         FEATURE1         FEATURE2  TARGET
0   1  feature1_value1  feature2_value1     0.5
1   2  feature1_value2  feature2_value2     1.5


/tmp/ipykernel_208/31309882.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_data = pd.read_sql(fetch_test_query, conn)


Test Data:
   ID         FEATURE1         FEATURE2  TARGET
0   1  feature1_value1  feature2_value1     0.7
1   2  feature1_value2  feature2_value2     1.7


True

# Close the Connection
Close the connection to Snowflake using the close() method.

In [7]:
# Close the Connection

# Close the connection to Snowflake
conn.close()